## 1. Imports

In [2]:
import random
import math
import copy

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import os
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

import torch
from kan import KAN

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime

C:\Users\Gavriil\AppData\Local\Temp\ipykernel_13560\4041496906.py:27: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [3]:
#from tools import KAN_es
from algos import vector_pred_skl, vector_pred_NN, vector_pred_KAN, multi_exp

## 2. Data preparation

### 2.1. Loading data

In [4]:
'''
initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_vld.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_vld.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_vld.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
'''
# !Some troubles with *.csv naming!

initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_pro.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_pro.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_pro.csv")

In [5]:
data_col_names = initial_trn_data_udus.columns

### 2.2. Scaling data

In [6]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(pd.concat([initial_trn_data_fdfs,
                         initial_trn_data_udfs, 
                         initial_trn_data_udus],
                        axis=0, sort=False, ignore_index=True))

MinMaxScaler()

In [6]:
'''
scaled_trn_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
scaled_vld_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
scaled_tst_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)

scaled_trn_data_udfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_udfs),columns=data_col_names)
scaled_vld_data_udfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_udfs),columns=data_col_names)
scaled_tst_data_udfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_udfs),columns=data_col_names)

scaled_trn_data_udus = pd.DataFrame(mm_scaler.transform(initial_trn_data_udus),columns=data_col_names)
scaled_vld_data_udus = pd.DataFrame(mm_scaler.transform(initial_vld_data_udus),columns=data_col_names)
scaled_tst_data_udus = pd.DataFrame(mm_scaler.transform(initial_tst_data_udus),columns=data_col_names)
'''
trn = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
vld = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
tst = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)


# 3. Experiment №1
* Input data: G, M, T, GMT.
* Models: Group Method Data Holding, Random Forest, Gradient Boosting, Multi-Layer Perceptron, Kolmogorov-Arnold Network.
* Single mode predicting: seperate model per each predicting depth.

In [ ]:
NUM_ITER = 3

In [7]:
from sklearn.base import BaseEstimator
from gmdh import Ria

def skl_regr_wrap(class_model, default_init_kwargs = {}, default_fit_kwargs = {}):
    '''
    Wrapper for regressor classes with fit, predict methods. Makes regressor compatable with skl cross-validation.
    Returns new class.
    '''
    class MyEstimator(BaseEstimator):
        '''
        Provides Sci-kit learn compatable Regressor class from class model with fit, predict functions 
        '''
        def __init__(self, *, init_kwargs=default_init_kwargs, fit_kwargs=default_fit_kwargs, random_state=1):
            self.init_kwargs = init_kwargs
            self.fit_kwargs = fit_kwargs
            self.class_model = class_model

        def fit(self, X, y):
            self.model = class_model(**self.init_kwargs)
            self.model.fit(X=np.array(X), y=np.array(y), **self.fit_kwargs)
            self.is_fitted_ = True
            return self

        def predict(self, X):
            return self.model.predict(X=X)
        
        
    return MyEstimator

In [8]:
l_algos_names=['G_GMDH', 'M_GMDH', 'T_GMDH', 'GMT_GMDH',
               'G_RF', 'M_RF', 'T_RF', 'GMT_RF',
               'G_GB', 'M_GB', 'T_GB', 'GMT_GB',
               'G_MLP', 'M_MLP', 'T_MLP', 'GMT_MLP',
               'G_KAN', 'M_KAN', 'T_KAN', 'GMT_KAN']

l_algos=[vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl, 
         vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl, 
         vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl, 
         vector_pred_NN, vector_pred_NN, vector_pred_NN, vector_pred_NN,
         vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4*5)]

l_geophysical_method = ['G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4*5)]

l_kwargs=[{'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1))}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1))}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1))},
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1))},
          
          {'class_model': RandomForestRegressor}, {'class_model': RandomForestRegressor}, {'class_model': RandomForestRegressor}, {'class_model': RandomForestRegressor}, 
          
          {'class_model': GradientBoostingRegressor}, {'class_model': GradientBoostingRegressor}, {'class_model': GradientBoostingRegressor}, {'class_model': GradientBoostingRegressor}, 

          {'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          {'learning_rate': 0.1, 'momentum':0.5, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05},
          
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0},
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=NUM_ITER

In [ ]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

--- Processing G_GMDH
iter: 1


In [ ]:
full_df

In [ ]:
full_df.to_excel('full_metrics_1.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics_1.xlsx')
aggr_df

---

# GMDH experiments

In [50]:
from sklearn.base import BaseEstimator
from gmdh import Ria

def skl_regr_wrap(class_model, default_init_kwargs = {}, default_fit_kwargs = {}):
    '''
    Wrapper for regressor classes with fit, predict methods. Makes regressor compatable with skl cross-validation.
    Returns new class.
    '''
    class MyEstimator(BaseEstimator):
        '''
        Provides Sci-kit learn compatable Regressor class from class model with fit, predict functions 
        '''
        def __init__(self, *, init_kwargs=default_init_kwargs, fit_kwargs=default_fit_kwargs, random_state=1):
            self.init_kwargs = init_kwargs
            self.fit_kwargs = fit_kwargs
            self.class_model = class_model

        def fit(self, X, y):
            self.model = class_model(**self.init_kwargs)
            self.model.fit(X=np.array(X), y=np.array(y), **self.fit_kwargs)
            self.is_fitted_ = True
            return self

        def predict(self, X):
            return self.model.predict(X=X)
        
        
    return MyEstimator

In [52]:
l_algos_names=['G_GMDH', 'M_GMDH', 'T_GMDH']

#gmdh_model = skl_regr_wrap(class_model=Ria)(fit_kwargs = dict(k_best=15, p_average=1))

l_algos=[vector_pred_skl, vector_pred_skl, vector_pred_skl]

Data = (trn, vld, tst)

mult_data = [Data for i in range(3*1)]

l_geophysical_method = ['G', 'M', 'T']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(3*1)]

l_kwargs=[{'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1))}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1))}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1))}
          ]

l_metrics_names=['rmse', 'mae', 'mape', 'r2'] #['mae', 'rmse', 'r2', 'mape']

num_iter=NUM_ITER

In [56]:
full_df_1 = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

--- Processing G_GMDH
iter: 1
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 1
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 1


c:\Users\Gavriil\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 1
iter: 2
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 2
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 2


c:\Users\Gavriil\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 2
iter: 3
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 3
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 3


...

c:\Users\Gavriil\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 4
iter: 5
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 5
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 5


c:\Users\Gavriil\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 5
iter: 6
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 6
<class '__main__.skl_regr_wrap.<locals>.MyEstimator'> fitted with randomseed: 6


KeyboardInterrupt: 

In [ ]:
full_df_1

In [ ]:
full_df_1.to_excel('full_metrics_1_gmdh.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df_1 = full_df_1.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df_1.to_excel('aggr_metrics_1_gmdh.xlsx')
aggr_df_1